In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Flatten, Dense
from tensorflow.keras.models import Model

In [3]:
# Load data
data = pd.read_csv('/content/drive/My Drive/M2 GL/PFE/Data/hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing_v2.csv')
safa_data = pd.read_csv('/content/drive/My Drive/M2 GL/PFE/Data/safa_hisb_60_and_Al_fatihah_audio_with_transcript_and_MFCC_and_ahkam_indexing.csv')

In [4]:
export_dir = '/content/drive/My Drive/M2 GL/PFE/AI_models_v4'

In [5]:
abdul_basit = data[data['recitor_en'] == 'Abdul Basit']
yassin_aljazaery = data[data['recitor_en'] == 'Yassin Al Jazaery']
ibrahim_aldosary = data[data['recitor_en'] == 'Ibrahim_Aldosary']

In [6]:
splitted_data_info_np = np.empty((0, 6))
models_information_np = np.empty((0, 5))

In [7]:
def max_sequence_length_X_Y(data, tajweed_rule):
  X_raw = data['mfcc'].astype(str).tolist()
  Y_raw = data[tajweed_rule].astype(str).tolist()
  X = [tf.constant(eval(x)) for x in X_raw]
  Y = [tf.constant(eval(x)) for x in Y_raw]
  max_sequence_length_Y = max(len(seq) for seq in Y)
  max_sequence_length_X = max(len(seq) for seq in X)
  return max_sequence_length_X, max_sequence_length_Y

In [8]:
def data_preparation(reciter_data, tajweed_rule, max_X, max_Y):
  # Extract 'mfcc' and tajweed_rule columns as lists of strings
  X_raw = reciter_data['mfcc'].astype(str).tolist()
  Y_raw = reciter_data[tajweed_rule].astype(str).tolist()

  # Preprocess the input data (X)
  X = [tf.constant(eval(x)) for x in X_raw]
  Y = [tf.constant(eval(x)) for x in Y_raw]

  # Pad sequences in Y and X to ensure all have the same length
  Y_padded = tf.keras.preprocessing.sequence.pad_sequences(Y, maxlen=max_Y, padding='post', dtype='int32', value=-1)
  X_padded = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=max_X, padding='post', dtype='float32')

  # Split the data into training and testing sets
  X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y_padded, test_size=0.2, random_state=10)
  return X_train, X_test, Y_train, Y_test

In [9]:
def tajweed_rule_model(reciter1, reciter2, reciter3, not_exp, tajweed_rule):
  global splitted_data_info_np, models_information_np, data

  max_X, max_Y = max_sequence_length_X_Y(data, tajweed_rule)

  # data preparation
  reciter1_X_train, reciter1_X_test, reciter1_Y_train, reciter1_Y_test = data_preparation(reciter1, tajweed_rule, max_X, max_Y)
  reciter2_X_train, reciter2_X_test, reciter2_Y_train, reciter2_Y_test = data_preparation(reciter2, tajweed_rule, max_X, max_Y)
  reciter3_X_train, reciter3_X_test, reciter3_Y_train, reciter3_Y_test = data_preparation(reciter3, tajweed_rule, max_X, max_Y)
  not_exp_X_train, not_exp_X_test, not_exp_Y_train, not_exp_Y_test = data_preparation(not_exp, tajweed_rule, max_X, max_Y)


  # Update splitted_data_info with information about each reciter
  for reciter_X_train, reciter_X_test, reciter_Y_train, reciter_Y_test, reciter_data in [
      (reciter1_X_train, reciter1_X_test, reciter1_Y_train, reciter1_Y_test, reciter1),
      (reciter2_X_train, reciter2_X_test, reciter2_Y_train, reciter2_Y_test, reciter2),
      (reciter3_X_train, reciter3_X_test, reciter3_Y_train, reciter3_Y_test, reciter3),
      (not_exp_X_train, not_exp_X_test, not_exp_Y_train, not_exp_Y_test, not_exp)]:


      splitted_data_info_np = np.append(splitted_data_info_np, [[
              tajweed_rule,
              reciter_data.iloc[0]['recitor_en'],
              len(reciter_X_train),
              len(reciter_X_test),
              len(reciter_Y_train),
              len(reciter_Y_test)
              ]], axis=0)

  # concatenate data
  # training data
  X_train = np.concatenate([reciter1_X_train, reciter2_X_train, reciter3_X_train, not_exp_X_train], axis=0)
  Y_train = np.concatenate([reciter1_Y_train, reciter2_Y_train, reciter3_Y_train, not_exp_Y_train], axis=0)

  # testing data
  X_test = np.concatenate([reciter1_X_test, reciter2_X_test, reciter3_X_test, not_exp_X_test], axis=0)
  Y_test = np.concatenate([reciter1_Y_test, reciter2_Y_test, reciter3_Y_test, not_exp_Y_test], axis=0)

  splitted_data_info_np = np.append(splitted_data_info_np, [[
          tajweed_rule,
          'all reciters',
          len(X_train),
          len(X_test),
          len(Y_train),
          len(Y_test)
          ]], axis=0)

  # Normalize input data by scaling each sequence individually
  scaler = StandardScaler()
  X_train_scaled = np.array([scaler.fit_transform(seq) for seq in X_train])
  X_test_scaled = np.array([scaler.transform(seq) for seq in X_test])

  # Define a simple neural network model
  input_shape = X_train_scaled[0].shape  # Shape of each mfcc sequence
  output_shape = Y_train.shape[1]  # Dimension of output (number of units in output layer)

  input_layer = Input(shape=input_shape)
  flatten_layer = Flatten()(input_layer)  # Flatten the sequence to a 1D vector
  hidden_layer = Dense(64, activation='relu')(flatten_layer)
  output_layer = Dense(output_shape, activation='linear')(hidden_layer)  # Define the output layer with the correct units

  model = Model(inputs=input_layer, outputs=output_layer)

  # Compile the model
  model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

  # Train the model
  model.fit(X_train_scaled, Y_train, epochs=50, batch_size=32, validation_split=0.1)

  #export the model
  model_filename = f'{tajweed_rule}_tajweed_rule_model'
  model_path = os.path.join(export_dir, model_filename)
  keras.models.save_model(model, model_path)

  # Make predictions on test data
  predictions = model.predict(X_test_scaled)

  # Evaluate the model with adjusted predictions
  predictions[predictions < 0] = -1
  predictions = np.round(predictions).astype('int32')
  loss, accuracy = model.evaluate(X_test_scaled, predictions)

  print(f"Test Loss: {loss:.4f}, Test accuracy : {accuracy:.4f}")
  models_information_np = np.append(models_information_np, [[
          model_filename,
          "{:.4f}".format(loss),
          "{:.4f}".format(accuracy),
          "{:.2f}".format(accuracy*100),
          model_path]], axis=0)

In [10]:
tajweed_rules = ['madd_6_Lazim', 'madd_246', 'madd_6', 'madd_2', 'Ikhfaa', 'Idgham', 'tafkhim', 'qalqala', 'imala']

In [12]:
for rule in tajweed_rules:
  tajweed_rule_model(abdul_basit, yassin_aljazaery, ibrahim_aldosary, safa_data, rule)

Epoch 1/50
27/27 [==============================] - 3s 86ms/step - loss: 89.8977 - accuracy: 0.8692 - val_loss: 2.9006 - val_accuracy: 0.9271
Epoch 2/50
27/27 [==============================] - 3s 117ms/step - loss: 11.7493 - accuracy: 0.9743 - val_loss: 2.8917 - val_accuracy: 0.9688
Epoch 3/50
27/27 [==============================] - 3s 115ms/step - loss: 10.0221 - accuracy: 0.9918 - val_loss: 2.8844 - val_accuracy: 0.9896
Epoch 4/50
27/27 [==============================] - 2s 73ms/step - loss: 9.7275 - accuracy: 0.9930 - val_loss: 2.8760 - val_accuracy: 0.9896
Epoch 5/50
27/27 [==============================] - 2s 73ms/step - loss: 9.6827 - accuracy: 0.9930 - val_loss: 2.8662 - val_accuracy: 0.9896
Epoch 6/50
27/27 [==============================] - 2s 72ms/step - loss: 9.6042 - accuracy: 0.9930 - val_loss: 2.8569 - val_accuracy: 0.9896
Epoch 7/50
27/27 [==============================] - 2s 71ms/step - loss: 9.5427 - accuracy: 0.9930 - val_loss: 2.8450 - val_accuracy: 0.9896
Epoch 8/

In [13]:
def print_model_summary(loaded_model, tajweed_rule):
  print(f'******* Tajweed rule {tajweed_rule} model *******')
  loaded_model.summary()
  print('\n')

In [14]:
for rule in tajweed_rules:
    model_filename = f'{rule}_tajweed_rule_model'
    model_path = os.path.join(export_dir, model_filename)

    # Load the saved model
    loaded_model = tf.keras.models.load_model(model_path)

    print_model_summary(loaded_model, rule)

******* Tajweed rule madd_6_Lazim model *******
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 6267, 13)]        0         
                                                                 
 flatten (Flatten)           (None, 81471)             0         
                                                                 
 dense (Dense)               (None, 64)                5214208   
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 5214338 (19.89 MB)
Trainable params: 5214338 (19.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


******* Tajweed rule madd_246 model *******
Model: "model_1"
__________________________________________

In [15]:
# how data is splitted
columns1 = ['tajweed_rule', 'data_of', 'X_train_nb_samples', 'X_test_nb_samples', 'Y_train_nb_samples', 'X_test_nb_samples']
splitted_data_info = pd.DataFrame(data=splitted_data_info_np, columns=columns1)

# save models information
columns2 = ['Model', 'Loss', 'Accuracy', 'Accuracy %', 'Path_to_the_model']
models_information = pd.DataFrame(data=models_information_np, columns=columns2)

In [16]:
splitted_data_info

,tajweed_rule,data_of,X_train_nb_samples,X_test_nb_samples,Y_train_nb_samples,X_test_nb_samples
0,madd_6_Lazim,Abdul Basit,238,60,238,60
1,madd_6_Lazim,Yassin Al Jazaery,238,60,238,60
2,madd_6_Lazim,Ibrahim_Aldosary,238,60,238,60
3,madd_6_Lazim,safa,238,60,238,60
4,madd_6_Lazim,all reciters,952,240,952,240
5,madd_246,Abdul Basit,238,60,238,60
6,madd_246,Yassin Al Jazaery,238,60,238,60
7,madd_246,Ibrahim_Aldosary,238,60,238,60
8,madd_246,safa,238,60,238,60
9,madd_246,all reciters,952,240,952,240


In [17]:
models_information

,Model,Loss,Accuracy,Accuracy %,Path_to_the_model
0,madd_6_Lazim_tajweed_rule_model,116.9364,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
1,madd_246_tajweed_rule_model,654.1515,0.9917,99.17,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
2,madd_6_tajweed_rule_model,126.5052,0.9625,96.25,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
3,madd_2_tajweed_rule_model,45.7938,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
4,Ikhfaa_tajweed_rule_model,507.0646,0.9833,98.33,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
5,Idgham_tajweed_rule_model,231.7475,1.0000,100.00,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
6,tafkhim_tajweed_rule_model,414.3999,0.9417,94.17,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
7,qalqala_tajweed_rule_model,1053.6315,0.9708,97.08,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
8,imala_tajweed_rule_model,690.7078,0.7875,78.75,/content/drive/My Drive/M2 GL/PFE/AI_models_v4...
